In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import json 
import argparse 
import sys

In [3]:
from rmab.simulator import RMABSimulator, random_valid_transition, random_valid_transition_round_down, synthetic_transition_small_window
from rmab.uc_whittle import UCWhittle
from rmab.ucw_value import UCWhittle_value
from rmab.baselines import optimal_policy, random_policy, WIQL
from rmab.fr_dynamics import get_all_transitions
from rmab.utils import get_save_path, delete_duplicate_results


In [4]:
is_jupyter = 'ipykernel' in sys.modules

In [22]:
if is_jupyter: 
    seed        = 42
    n_arms      = 8
    budget      = 3
    discount    = 0.9
    alpha       = 3 
    n_episodes  = 30
    episode_len = 20
    n_epochs    = 10
    save_name = 'results_all_agents'
    match_prob = 0.25
    save_with_date = True 
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--n_arms',         '-N', help='num beneficiaries (arms)', type=int, default=8)
    parser.add_argument('--episode_len',    '-H', help='episode length', type=int, default=20)
    parser.add_argument('--n_episodes',     '-T', help='num episodes', type=int, default=30)
    parser.add_argument('--budget',         '-B', help='budget', type=int, default=3)
    parser.add_argument('--n_epochs',       '-E', help='number of epochs (num_repeats)', type=int, default=10)
    parser.add_argument('--discount',       '-d', help='discount factor', type=float, default=0.9)
    parser.add_argument('--alpha',          '-a', help='alpha: for conf radius', type=float, default=3)
    parser.add_argument('--seed',           '-s', help='random seed', type=int, default=42)
    parser.add_argument('--save_name',      '-n', help='save name', type=str, default='results')
    parser.add_argument('--use_date', action='store_true')

    args = parser.parse_args()

    n_arms      = args.n_arms
    budget      = args.budget
    discount    = args.discount
    alpha       = args.alpha 
    seed        = args.seed
    n_episodes  = args.n_episodes
    episode_len = args.episode_len
    n_epochs    = args.n_epochs
    save_name   = args.save_name 
    save_with_date = args.use_date 



In [6]:
n_states = 2
n_actions = 2

In [7]:
all_population_size = 100 # number of random arms to generate
all_transitions = get_all_transitions(all_population_size)

In [8]:
all_transitions.shape

(100, 2, 2, 2)

In [9]:
all_features = np.arange(all_population_size)

In [10]:
simulator = RMABSimulator(all_population_size, all_features, all_transitions,
            n_arms, episode_len, n_epochs, n_episodes, budget, number_states=n_states, reward_style='match',match_probability=match_prob)

acting should always be good! 0.000 < 0.044
acting should always be good! 0.000 < 0.162
acting should always be good! 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [66 49 92  1 98 21 96 47]
cohort [73 89 44 62 98 65  3 39]
cohort [47 37 51 42 94 90  2 52]
cohort [32 46 65 33 66 25 94 83]
cohort [16 15 49 13 92 85 74 28]
cohort [49 46  2 64 13 27 15 48]
cohort [39 46 61 40 96 91 20 72]
cohort [43 74 70 25 66 56 96 33]
cohort [60 63 93  5 58  8 47 70]
cohort [ 3 35 49 17 86  9 94 23]


In [11]:
np.random.seed(seed)
random.seed(seed)
random_rewards = random_policy(simulator, n_episodes, n_epochs)

first state [1 0 1 0 0 1 1 0]
instance 0, ep 1, state [1 0 1 0 1 0 1 1]
instance 0, ep 2, state [1 0 1 1 0 0 1 0]
instance 0, ep 3, state [1 0 1 1 1 1 0 0]
instance 0, ep 4, state [0 1 1 0 1 1 0 0]
instance 0, ep 5, state [1 0 0 0 0 0 0 0]
instance 0, ep 6, state [0 1 1 1 1 1 0 1]
instance 0, ep 7, state [0 0 0 1 1 1 0 1]
instance 0, ep 8, state [0 1 0 1 1 1 0 0]
instance 0, ep 9, state [0 0 0 1 0 0 0 0]
instance 0, ep 10, state [1 1 0 1 1 0 1 0]
instance 0, ep 11, state [1 1 0 1 1 0 1 1]
instance 0, ep 12, state [1 0 0 0 0 0 1 1]
instance 0, ep 13, state [0 1 0 0 0 1 1 0]
instance 0, ep 14, state [0 1 0 0 1 1 0 1]
instance 0, ep 15, state [0 1 1 0 0 0 1 0]
instance 0, ep 16, state [1 1 0 0 0 0 1 0]
instance 0, ep 17, state [0 1 0 0 1 0 1 1]
instance 0, ep 18, state [0 0 1 1 0 0 0 0]
instance 0, ep 19, state [1 1 0 0 0 0 0 1]
instance 0, ep 20, state [0 0 0 0 1 0 0 0]
instance 0, ep 21, state [0 0 1 1 1 0 1 1]
instance 0, ep 22, state [0 1 1 0 1 0 0 0]
instance 0, ep 23, state [1 0 0 1

In [12]:
np.random.seed(seed)
random.seed(seed)
optimal_reward = optimal_policy(simulator, n_episodes, n_epochs, discount)

first state [1 0 1 0 0 1 1 0]
   state [1 0 1 0 0 1 1 0] state_WI [  0.34   0.95   0.18  -0.04   0.59 -10.   -10.     0.68] sorted [1 7 4]
   state [0 1 1 0 0 1 0 0] state_WI [  0.32   0.3    0.18  -0.04   0.59 -10.     0.92   0.68] sorted [6 7 4]
   state [0 1 1 0 1 1 1 0] state_WI [  0.32   0.3    0.18  -0.04   0.4  -10.   -10.     0.68] sorted [7 4 0]
   state [0 0 1 0 1 0 1 1] state_WI [  0.32   0.95   0.18  -0.04   0.4    0.53 -10.   -10.  ] sorted [1 5 4]
   state [0 1 0 0 1 1 1 0] state_WI [  0.32   0.3    1.    -0.04   0.4  -10.   -10.     0.68] sorted [2 7 4]
   state [0 1 0 0 1 1 1 0] state_WI [  0.32   0.3    1.    -0.04   0.4  -10.   -10.     0.68] sorted [2 7 4]
   state [0 1 1 0 0 1 0 1] state_WI [  0.32   0.3    0.18  -0.04   0.59 -10.     0.92 -10.  ] sorted [6 4 0]
   state [1 0 0 1 1 0 1 1] state_WI [  0.34   0.95   1.    -0.15   0.4    0.53 -10.   -10.  ] sorted [2 1 5]
   state [0 1 1 0 1 1 1 1] state_WI [  0.32   0.3    0.18  -0.04   0.4  -10.   -10.   -10.  ] sort

In [15]:
np.random.seed(seed)
random.seed(seed)
wiql_rewards = WIQL(simulator, n_episodes, n_epochs)

first state [1 0 1 0 0 1 1 0]
instance 0, ep 1, state [1 0 1 0 1 0 1 1]
instance 0, ep 2, state [1 0 1 1 0 0 1 0]
instance 0, ep 3, state [1 0 1 1 1 1 0 0]
instance 0, ep 4, state [0 1 1 0 1 1 0 0]
instance 0, ep 5, state [1 0 0 0 0 0 0 0]
instance 0, ep 6, state [0 1 1 1 1 1 0 1]
instance 0, ep 7, state [0 0 0 1 1 1 0 1]
instance 0, ep 8, state [0 1 0 1 1 1 0 0]
instance 0, ep 9, state [0 0 0 1 0 0 0 0]
instance 0, ep 10, state [1 1 0 1 1 0 1 0]
instance 0, ep 11, state [1 1 0 1 1 0 1 1]
instance 0, ep 12, state [1 0 0 0 0 0 1 1]
instance 0, ep 13, state [0 1 0 0 0 1 1 0]
instance 0, ep 14, state [0 1 0 0 1 1 0 1]
instance 0, ep 15, state [0 1 1 0 0 0 1 0]
instance 0, ep 16, state [1 1 0 0 0 0 1 0]
instance 0, ep 17, state [0 1 0 0 1 0 1 1]
instance 0, ep 18, state [0 0 1 1 0 0 0 0]
instance 0, ep 19, state [1 1 0 0 0 0 0 1]
instance 0, ep 20, state [0 0 0 0 1 0 0 0]
instance 0, ep 21, state [0 0 1 1 1 0 1 1]
instance 0, ep 22, state [0 1 1 0 1 0 0 0]
instance 0, ep 23, state [1 0 0 1

In [16]:
np.random.seed(seed)
random.seed(seed)
ucw_extreme_rewards = UCWhittle(simulator, n_episodes, n_epochs, discount, alpha=alpha, method='extreme')

solving UCWhittle using method: extreme
first state [1 0 1 0 0 1 1 0]
instance 0, ep 1, state [1 0 1 0 1 0 1 1]
instance 0, ep 2, state [1 0 1 1 0 0 1 0]
instance 0, ep 3, state [1 0 1 1 1 1 0 0]
instance 0, ep 4, state [0 1 1 0 1 1 0 0]
instance 0, ep 5, state [1 0 0 0 0 0 0 0]
---------------------------------------------------
0 100  | a  [0 0 0 1 0 0 1 1]  | s'  [0 0 0 0 0 0 1 0]  | r  0.25    | WI  [0.899 0.899 0.899 0.9   0.899 0.899 0.899 0.9  ]
instance 0, ep 6, state [0 1 1 1 1 1 0 1]
instance 0, ep 7, state [0 0 0 1 1 1 0 1]
instance 0, ep 8, state [0 1 0 1 1 1 0 0]
instance 0, ep 9, state [0 0 0 1 0 0 0 0]
instance 0, ep 10, state [1 1 0 1 1 0 1 0]
---------------------------------------------------
0 200  | a  [0 0 0 0 0 1 1 1]  | s'  [0 0 0 0 0 1 1 1]  | r  0.578125    | WI  [0.899 0.899 0.899 0.899 0.899 0.9   0.899 0.9  ]
instance 0, ep 11, state [1 1 0 1 1 0 1 1]
instance 0, ep 12, state [1 0 0 0 0 0 1 1]
instance 0, ep 13, state [0 1 0 0 0 1 1 0]
instance 0, ep 14, sta

In [17]:
np.random.seed(seed)
random.seed(seed)
ucw_ucb_rewards = UCWhittle(simulator, n_episodes, n_epochs, discount, alpha=alpha, method='UCB')

solving UCWhittle using method: UCB
first state [1 0 1 0 0 1 1 0]
instance 0, ep 1, state [1 0 1 0 1 0 1 1]
instance 0, ep 2, state [1 0 1 1 0 0 1 0]
instance 0, ep 3, state [1 0 1 1 1 1 0 0]
instance 0, ep 4, state [0 1 1 0 1 1 0 0]
instance 0, ep 5, state [1 0 0 0 0 0 0 0]
---------------------------------------------------
0 100  | a  [0 0 0 1 0 0 1 1]  | s'  [1 0 1 0 0 0 0 1]  | r  0.578125    | WI  [-0.    -0.    -0.     0.116 -0.    -0.    -0.    -0.   ]
instance 0, ep 6, state [0 1 1 1 1 1 0 1]
instance 0, ep 7, state [0 0 0 1 1 1 0 1]
instance 0, ep 8, state [0 1 0 1 1 1 0 0]
instance 0, ep 9, state [0 0 0 1 0 0 0 0]
instance 0, ep 10, state [1 1 0 1 1 0 1 0]
---------------------------------------------------
0 200  | a  [0 1 0 1 0 0 0 1]  | s'  [0 0 1 0 0 0 1 1]  | r  0.578125    | WI  [-0.     0.004 -0.     0.269 -0.    -0.    -0.    -0.   ]
instance 0, ep 11, state [1 1 0 1 1 0 1 1]
instance 0, ep 12, state [1 0 0 0 0 0 1 1]
instance 0, ep 13, state [0 1 0 0 0 1 1 0]
instan

In [18]:
mean_rewards = {'random_rewards': np.mean(random_rewards), 
 'optimal_rewards': np.mean(optimal_reward), 
 'wiql_rewards': np.mean(wiql_rewards), 
 'extreme_rewards': np.mean(ucw_extreme_rewards), 
 'ucb_rewards': np.mean(ucw_ucb_rewards)}

In [19]:
std_rewards = {'random_rewards': np.std(random_rewards), 
 'optimal_rewards': np.std(optimal_reward), 
 'wiql_rewards': np.std(wiql_rewards), 
 'extreme_rewards': np.std(ucw_extreme_rewards), 
 'ucb_rewards': np.std(ucw_ucb_rewards)}

In [20]:
data = {
    'mean_reward': mean_rewards, 
    'std_reward': std_rewards,
    'parameters': 
        {'seed'      : seed,
        'n_arms'    : n_arms,
        'budget'    : budget,
        'discount'  : discount, 
        'alpha'     : alpha, 
        'n_episodes': n_episodes, 
        'episode_len': episode_len, 
        'n_epochs'  : n_epochs, 
        'match_prob': match_prob} 
}

In [23]:
save_path = get_save_path('matching',save_name,seed,use_date=save_with_date)

In [25]:
delete_duplicate_results('matching',save_name,data)

In [26]:
json.dump(data,open('../results/'+save_path,'w'))